In [179]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import OneHotEncoder

In [171]:
df = pd.read_csv('../data_saved/data_mat.csv')
df_gps = pd.read_csv('../data/cities_coordinates_gps.csv', index_col = 0).T

df.head()

,id_Location,id_Date,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,...,Climate,Year,Month,Season,WindGustDir_cos,WindGustDir_sin,WindDir9am_cos,WindDir9am_sin,WindDir3pm_cos,WindDir3pm_sin
0,Adelaide,2008-07-01,2008-07-01,Adelaide,8.8,15.7,5.0,48.0,13.0,15.0,...,Temperate,2008,July,Winter,7.071068e-01,-0.707107,-0.707107,-0.707107,-1.836970e-16,-1.000000
1,Adelaide,2008-07-02,2008-07-02,Adelaide,12.7,15.8,0.8,35.0,13.0,15.0,...,Temperate,2008,July,Winter,-7.071068e-01,-0.707107,-0.923880,-0.382683,-7.071068e-01,-0.707107
2,Adelaide,2008-07-03,2008-07-03,Adelaide,6.2,15.1,0.0,20.0,2.0,11.0,...,Temperate,2008,July,Winter,-1.836970e-16,-1.000000,0.923880,0.382683,-7.071068e-01,-0.707107
3,Adelaide,2008-07-04,2008-07-04,Adelaide,5.3,15.9,0.0,30.0,6.0,13.0,...,Temperate,2008,July,Winter,9.238795e-01,0.382683,0.923880,0.382683,7.071068e-01,0.707107
4,Adelaide,2008-07-07,2008-07-07,Adelaide,7.6,11.2,16.2,46.0,17.0,13.0,...,Temperate,2008,July,Winter,-3.826834e-01,-0.923880,0.382683,-0.923880,-7.071068e-01,-0.707107


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135600 entries, 0 to 135599
Data columns (total 28 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id_Location      135600 non-null  object 
 1   id_Date          135600 non-null  object 
 2   Date             135600 non-null  object 
 3   Location         135600 non-null  object 
 4   MinTemp          135600 non-null  float64
 5   MaxTemp          135600 non-null  float64
 6   Rainfall         135600 non-null  float64
 7   WindGustSpeed    135600 non-null  float64
 8   WindSpeed9am     135600 non-null  float64
 9   WindSpeed3pm     135600 non-null  float64
 10  Humidity9am      135600 non-null  float64
 11  Humidity3pm      135600 non-null  float64
 12  Pressure9am      135600 non-null  float64
 13  Pressure3pm      135600 non-null  float64
 14  Temp9am          135600 non-null  float64
 15  Temp3pm          135600 non-null  float64
 16  RainToday        135600 non-null  floa

# encodage des locations

In [173]:
df['gps_lat'] = df['Location'].map(lambda x : df_gps.loc[x, 'lat'])
df['gps_lon'] = df['Location'].map(lambda x : df_gps.loc[x, 'long'])

df['sin_lat'] = df['gps_lat'].apply(lambda x : np.sin(np.radians(x)))
df['cos_lat'] = df['gps_lat'].apply(lambda x : np.cos(np.radians(x)))

df['sin_lon'] = df['gps_lon'].apply(lambda x : np.round(np.sin(np.radians(x)), 6))
df['cos_lon'] = df['gps_lon'].apply(lambda x : np.round(np.cos(np.radians(x)), 6))

df = df.drop(columns = ['Location', 'id_Location', 'gps_lat', 'gps_lon'])

# encodage Month

In [174]:
dict_mois = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

df['Month'] = df['Month'].map(dict_mois)

df['sin_month'] = df['Month'].apply(lambda x : np.sin((2 * np.pi) * (( x - 1 ) / 12)))
df['cos_month'] = df['Month'].apply(lambda x : np.cos((2 * np.pi) * (( x - 1 ) / 12)))


df = df.drop(columns = 'Month')

# encodage season

In [175]:
df['Season'].unique()

array(['Winter', 'Spring', 'Summer', 'Autumn'], dtype=object)

In [176]:
dict_season = {
    'Spring' : 0,
    'Summer' : 1,
    'Autumn' : 2,
    'Winter' : 3
}

df['Season'] = df['Season'].map(dict_season)

df['sin_season'] = df['Season'].apply(lambda x : round(np.sin((2 * np.pi) * (x / 4))), 1)
df['cos_season'] = df['Season'].apply(lambda x : round(np.cos((2 * np.pi) * ( x / 4))), 1)

df = df.drop(columns = 'Season')

/tmp/ipykernel_9657/907855078.py:10: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['sin_season'] = df['Season'].apply(lambda x : round(np.sin((2 * np.pi) * (x / 4))), 1)
/tmp/ipykernel_9657/907855078.py:11: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['cos_season'] = df['Season'].apply(lambda x : round(np.cos((2 * np.pi) * ( x / 4))), 1)


# encodage Climate

In [197]:
ohe = OneHotEncoder(sparse_output = False)

climate_transform = ohe.fit_transform(df[['Climate']])
df_climate_ohe = pd.DataFrame(climate_transform, columns = ohe.get_feature_names_out())

df = pd.concat([df, df_climate_ohe], axis = 1)

df = df.drop(columns = 'Climate')